# Interactive PCA 

In case there is an already existing PCA and dataset skip all the steps except the last one, that is the interactive part

## Load all the modules

In [1]:
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D

import torch
import sys
from torch.utils.data import DataLoader
from torchvision import transforms
import seaborn as sns
import pandas as pd
import os
import numpy as np
import plotly.express as px
import pytorch_lightning as pl

import re
import warnings
from copy import deepcopy

# Custom functions/classes
path_to_module = './'  # Path where all the .py files are, relative to the notebook folder
sys.path.append(path_to_module)

from class_dataset import RandomCrop, Subtract, ToTensor, myDataset
from load_data import DataProcesser
from utils_app import frange, model_output_app
import results_model
from utils import model_output

ModuleNotFoundError: No module named 'torch'

## Set the files and variables

It is best to leave most variables as they are. Important is to set the paths to the zip archive with the data and the .pytorch model file. 

A name needs to be given to the analysis, this is just to store all results of one pca in a separate file to prevent files being overwritten. 

In [ ]:
myseed = 7
torch.manual_seed(myseed)
torch.cuda.manual_seed(myseed)
np.random.seed(myseed)

# Inputs
data_file = './ERKH/ERKH.zip'
model_file = './model/ERKKTR_model.pytorch'

# Name of the output files
name = 'ERKH'


# Leave at None for automatic detection
start_time = None
end_time = None
measurement = None
selected_classes = None
batch_sz = 2048  # set as high as GPU memory can handle for speed up, automatic detection not possible

rand_crop = True
set_to_project = 'all'  # one of ['all', 'train', 'validation', 'test']


# Leave this, more than 2 principal components are not currently possible. I was too ambitious
n_pca = 2

## Load the model

In [ ]:
# Model Loading

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
net = torch.load(model_file) if torch.cuda.is_available() else torch.load(model_file, map_location='cpu')
net.eval()
net.double()
net = net.to(device)
length = net.length
print('LENGTH IS ', length)

# ----------------------------------------------------------------------------------------------------------------------

# Data Loading, Subsetting, Preprocessing
perc_selected_ids = 1  # Select only percentile of all trajectories, not always useful to project them all and slow
data = DataProcesser(data_file, datatable=False)
measurement = data.detect_groups_times()['groups'] if measurement is None else measurement
start_time = data.detect_groups_times()['times'][0] if start_time is None else start_time
end_time = data.detect_groups_times()['times'][1] if end_time is None else end_time
data.subset(sel_groups=measurement, start_time=start_time, end_time=end_time)

data.get_stats()
data.split_sets()
classes = tuple(data.classes.iloc[:,1])
dict_classes = data.classes[data.col_classname]

# Check that the measurements columns are numeric, if not try to convert to float64
cols_to_check = '^(?:{})'.format('|'.join(measurement))  # ?: for non-capturing group
cols_to_check = data.dataset.columns.values[data.dataset.columns.str.contains(cols_to_check)]
cols_to_change = [(s,t) for s,t in zip(cols_to_check, data.dataset.dtypes[cols_to_check]) if not pd.api.types.is_numeric_dtype(data.dataset[s])]
if len(cols_to_change) > 0:
    warnings.warn('Some measurements columns are not of numeric type. Attempting to convert the columns to float64 type. List of problematic columns: {}'.format(cols_to_change))
    try:
        cols_dict = {s[0]:'float64' for s in cols_to_change}
        data.dataset = data.dataset.astype(cols_dict)
    except ValueError:
        warnings.warn('Conversion to float failed for at least one column.')

data.get_stats()
if selected_classes is not None:
    data.dataset = data.dataset[data.dataset[data.col_class].isin(selected_classes)]
# Suppress the warning that data were not processed, irrelevant for the app
with warnings.catch_warnings():
    warnings.simplefilter('ignore')
    data.split_sets(which='dataset')

print('Start time: {}; End time: {}; Measurement: {}'.format(start_time, end_time, measurement))

# ----------------------------------------------------------------------------------------------------------------------
# df used to plot measurements
assert set_to_project in ['all', 'train', 'validation', 'test']
if set_to_project == 'all':
    df = deepcopy(data.dataset)
elif set_to_project == 'train':
    df = deepcopy(data.train_set)
elif set_to_project == 'validation':
    df = deepcopy(data.validation_set)
elif set_to_project == 'test':
    df = deepcopy(data.test_set)
print('Size of raw dataframe: {}'.format(df.shape))
df.rename(columns={data.col_id: 'ID', data.col_class: 'Class'}, inplace = True)
selected_ids = np.random.choice(df.loc[:,'ID'].unique(), round(perc_selected_ids * df.shape[0]), replace=False)
df = df[df['ID'].isin(selected_ids)]
print('Size of selected dataframe: {}'.format(df.shape))

if batch_sz == -1:
    batch_sz = round(df.shape[0]/10)
print('Batch size: {}'.format(batch_sz))
assert batch_sz <= df.shape[0]
# Split for each measurement, melt and append.
ldf = []
for meas in measurement:
    col_meas = [i for i in df.columns if re.match('^{}_'.format(meas), i)]
    temp = df[['ID', 'Class'] + col_meas].melt(['ID', 'Class'])
    temp['Time'] = temp['variable'].str.extract('([0-9]+$)')
    temp['variable'] = temp['variable'].str.replace('_[0-9]+$', '', regex=True)
    ldf.append(temp)
df = pd.concat(ldf)
del temp
del ldf
df.sort_values(['ID', 'Time'])

# ----------------------------------------------------------------------------------------------------------------------
# Prepare dataloader for t-SNE, set high batch_size to speed up
subtract_numbers = [data.stats['mu'][meas]['train'] for meas in measurement]
if rand_crop:
    ls_transforms = transforms.Compose([
        Subtract(subtract_numbers),
        RandomCrop(output_size=length, ignore_na_tails=True, export_crop_pos=True),
        ToTensor()])
else:
    ls_transforms = transforms.Compose([
        Subtract(subtract_numbers),
        ToTensor()])

mydataset = myDataset(dataset=data.dataset[data.dataset['ID'].isin(selected_ids)], transform=ls_transforms)
mydataloader = DataLoader(dataset=mydataset,
                          batch_size=batch_sz,
                          shuffle=False,
                          drop_last=False)

# ----------------------------------------------------------------------------------------------------------------------
# Classes object definition
classes = tuple(data.classes[data.col_classname])
classes_col = data.classes[data.col_classname]
if selected_classes is not None:
    classes = [i for i in classes if i
               in list(data.classes[data.classes[data.col_class].isin(selected_classes)][data.col_classname])]
    classes = tuple(classes)
    classes_dict = data.classes[data.classes[data.col_class].isin(selected_classes)].to_dict()[data.col_classname]
else:
    classes_dict = data.classes.to_dict()[data.col_classname]

net.batch_size = batch_sz  # Learn representations over the whole dataset at once if equal to dataset length

## PCA with all values

In [ ]:
model = net
dataloader = mydataloader

df_out = model_output_app(model, dataloader, export_prob=True, export_feat=True, device=device, export_crop_pos=rand_crop)
df_out['Class'].replace(classes_col, inplace=True)
feat_cols = [i for i in df_out.columns if i.startswith('Feat_')]
feature_blobs_array = np.array(df_out[feat_cols])
pca = PCA(n_components=n_pca)
pca_original = pca.fit_transform(feature_blobs_array)

out_dir = './results_' + name + '/pca'
if not os.path.exists(out_dir):
    os.makedirs(out_dir)


df_raw = pd.DataFrame(pca_original)
df_original = df_raw.join(df_out)
df_original.to_csv(out_dir + '/pca_all.csv', index=False)
df_original = pd.DataFrame(df_original)

Nclasses = len(np.unique(df_original['Class']))
cmap = plt.cm.get_cmap('hsv')


label_color_dict = {label: cmap(np.linspace(0.2,1,Nclasses))[idx] for idx, label in enumerate(np.unique(df_original['Class']))}
#colors = [cmap(label_color_dict[label]) for label in df_original['Class']]

colors = [label_color_dict[label] for label in df_original['Class']]
#colors = df_original['Class'].astype(str).map(colors)


plt.scatter(df_original[0], df_original[1], alpha=0.1, c=colors)
custom_lines = [Line2D([0], [0], color=cmap(np.linspace(0.2,1,Nclasses))[i], lw=4) for i, cl in enumerate(cmap(np.linspace(0,1,Nclasses)))]
#custom_lines = [Line2D([0], [0], color=label_color_dict[cl], lw=4) for cl in label_color_dict.keys()]
plt.legend(custom_lines, label_color_dict.keys(), loc='upper right')

# Add the axis labels
plt.xlabel('PC 1 (%.2f%%)' % (pca.explained_variance_ratio_[0]*100))
plt.ylabel('PC 2 (%.2f%%)' % (pca.explained_variance_ratio_[1]*100))
plt.title('PCA-plot')
plt.savefig(out_dir + '/pca_all.pdf', format='pdf')
# Close the plot
plt.close()

## Set variables for the partial PCAs

Change the percentile to something mangable for the interactive PCA. Too many datapoints and it might crash. 

The threshold_confidence is the lowest value that the uncorrelated trajectories can take. This is to identify what a realistic trajectory in each class should look like. Setting it around 75% would be good, that way the picked trajectories not too clean but still representative. But of course the threshold might depend on if you think your model is over- or underfitted. An overfitted model might benefit from a lower confidence and an underfitted one from a higher one. 

In [ ]:
# Minimal confidence for uncorrelated prototypes
perc_selected_ids = 0.01  # Select only percentile of all trajectories, not always useful to project them all and slow
threshold_confidence = 0.75


# Leave the rest as is
# Helper functions for plotting
col_id = data.col_id
col_class = data.col_class
col_classname = data.col_classname


# Convert percentile to actuall number of datapoints
length_data = len(pd.DataFrame(df_original))
npoint = round(perc_selected_ids * length_data)
ntop = npoint
nworst = npoint
nuncor = npoint
nrandom = npoint

## PCA with the top class trajectories

In [ ]:
# ### Top trajectories per class
tops = results_model.top_confidence_perclass(model, dataloader, n=ntop, labels_classes=dict_classes)

df_tops = df_original.loc[df_original['ID'].isin(tops['ID'])]
df_tops.to_csv(out_dir + '/pca_tops_' + str(config_pca['perc_selected_ids']) + '.csv', index=False)

colors = [label_color_dict[label] for label in df_tops['Class']]
plt.scatter(df_tops[0], df_tops[1], alpha=0.1, c=colors)

custom_lines = [Line2D([0], [0], color=cmap(np.linspace(0.2,1,Nclasses))[i], lw=4) for i, cl in enumerate(cmap(np.linspace(0.2,1,Nclasses)))]
plt.legend(custom_lines, label_color_dict.keys(), loc='upper right')

# Add the axis labels
plt.xlabel('PC 1 (%.2f%%)' % (pca.explained_variance_ratio_[0]*100))
plt.ylabel('PC 2 (%.2f%%)' % (pca.explained_variance_ratio_[1]*100))
plt.title('PCA-plot tops')
plt.savefig(out_dir + '/pca_tops_' + str(config_pca['perc_selected_ids']) + '.pdf', format='pdf')
# Close the plot
plt.close()

## PCA with the least correlated class trajectories

In [ ]:
# ### Least correlated set per class
uncorr = results_model.least_correlated_set(model, dataloader, n=nuncor, labels_classes=dict_classes, threshold_confidence=threshold_confidence)

df_uncorr = df_original.loc[df_original['ID'].isin(uncorr['ID'])]
df_uncorr.to_csv(out_dir + '/pca_uncorr_' + str(config_pca['perc_selected_ids']) + '.csv', index=False)

colors = [label_color_dict[label] for label in df_uncorr['Class']]
plt.scatter(df_uncorr[0], df_uncorr[1], alpha=0.1, c=colors)

custom_lines = [Line2D([0], [0], color=cmap(np.linspace(0.2,1,Nclasses))[i], lw=4) for i, cl in enumerate(cmap(np.linspace(0.2,1,Nclasses)))]
plt.legend(custom_lines, label_color_dict.keys(), loc='upper right')

# Add the axis labels
plt.xlabel('PC 1 (%.2f%%)' % (pca.explained_variance_ratio_[0]*100))
plt.ylabel('PC 2 (%.2f%%)' % (pca.explained_variance_ratio_[1]*100))
plt.title('PCA-plot uncorr')
plt.savefig(out_dir + '/pca_uncorr_' + str(config_pca['perc_selected_ids']) + '.pdf', format='pdf')
# Close the plot
plt.close()

## PCA with the worst class trajectories

In [ ]:
# ### Worst trajectory per class
worsts = results_model.worst_classification_perclass(model, dataloader, n=nworst, labels_classes=dict_classes)

df_worsts = df_original.loc[df_original['ID'].isin(worsts['ID'])]
df_worsts.to_csv(out_dir + '/pca_worsts_' + str(perc_selected_ids) + '.csv', index=False)

colors = [label_color_dict[label] for label in df_worsts['Class']]
plt.scatter(df_worsts[0], df_worsts[1], alpha=0.1, c=colors)

custom_lines = [Line2D([0], [0], color=cmap(np.linspace(0.2,1,Nclasses))[i], lw=4) for i, cl in enumerate(cmap(np.linspace(0.2,1,Nclasses)))]
plt.legend(custom_lines, label_color_dict.keys(), loc='upper right')

# Add the axis labels
plt.xlabel('PC 1 (%.2f%%)' % (pca.explained_variance_ratio_[0]*100))
plt.ylabel('PC 2 (%.2f%%)' % (pca.explained_variance_ratio_[1]*100))
plt.title('PCA-plot worst')
plt.savefig(out_dir + '/pca_worsts_' + str(perc_selected_ids) + '.pdf', format='pdf')
# Close the plot
plt.close()


## PCA with random class trajectories

In [ ]:
# ### Random sample
# Get a random sample of trajectories.
randoms_ids = []
for classe in data.validation_set['class'].unique():
    randoms_ids += list(data.dataset.loc[data.dataset['class']==classe]['ID'].sample(nrandom))
randoms = model_output(model, dataloader)
randoms = randoms.loc[randoms['ID'].isin(randoms_ids)]

df_randoms = df_original.loc[df_original['ID'].isin(randoms['ID'])]
df_randoms.to_csv(out_dir + '/pca_randoms_' + str(perc_selected_ids) + '.csv', index=False)

colors = [label_color_dict[label] for label in df_randoms['Class']]
plt.scatter(df_randoms[0], df_randoms[1], alpha=0.1, c=colors)

custom_lines = [Line2D([0], [0], color=cmap(np.linspace(0.2,1,Nclasses))[i], lw=4) for i, cl in enumerate(cmap(np.linspace(0.2,1,Nclasses)))]
plt.legend(custom_lines, label_color_dict.keys(), loc='upper right')
# Add the axis labels
plt.xlabel('PC 1 (%.2f%%)' % (pca.explained_variance_ratio_[0]*100))
plt.ylabel('PC 2 (%.2f%%)' % (pca.explained_variance_ratio_[1]*100))
plt.title('PCA-plot randoms')
plt.savefig(out_dir + '/pca_randoms_' + str(perc_selected_ids) + '.pdf', format='pdf')
# Close the plot
plt.close()

## Interactive PCA with existing PCA results from above

This is where the interactive PCA will be displayed. There is two options to do this, with the data from above or with already existing results from previous calculations. 

If your doing it with the data from above, what would need to be changed which of the PCA you would want to use. Generally speaking, it is not recommended to use the full PCA since there will be too many datapoint and the interactive PCA might crash, so go any of the other 4 instead. 

Alternatively, change the input for the PCA results and the path to where the dataset is stored. The time_series is the dataset.csv file that was used for the previous parts. 

It is also possible to change the color scheme to any of the matplotlib schemes. Just change the name in the cmap variable. 

In [2]:
import pandas as pd
import plotly.express as px
import plotly.graph_objs as go
from plotly.subplots import make_subplots
from ipywidgets import Output, VBox
from IPython.display import display, HTML
import http.server
import socketserver
import threading
import webbrowser
import os
from collections import OrderedDict
import matplotlib.pyplot as plt
import numpy as np


### If you have already existing pca results and the dataset set their paths here manually
pca = pd.read_csv('../pca_original.csv', sep=',', index_col=False)
time_series = pd.read_csv('../ALLH_50intp.csv', sep=',', index_col=False)



### If you already did pcas from above just use this and chose the pca results that you would like to use

#time_series = data.datset

#pca = df_original   #Full pca
#pca = df_tops       #Only top results
#pca = df_uncorr     #Only uncorrelated results
#pca = df_worsts     #Only worst results
#pca = df_randoms    #Only random results


### Setting the colors scheme (any colorpalette from matplotlib is fine)
cmap = plt.cm.get_cmap('hsv')




#-------------------------------------------------------------------------------------------------------------------#
# From here on leave as is

Nclasses = len(np.unique(pca['Class']))
#color_dict = {label: cmap(np.linspace(0.2,1,Nclasses))[idx] for idx, label in enumerate(np.unique(pca['Class']))}
color_dict = {'WT': 'green', 'PIK3CA_H1047R': 'blue', 'ErbB2': 'lightgreen', 'Akt1_E17K': 'yellow', 'PTEN_del': 'purple', 'PIK3CA_E545K': 'lightblue'}


components = pca[0:2]

pca['colors'] = pca['Class'].apply(lambda x: color_dict.get(x))


# Make the graphs
fig = go.FigureWidget()
fig.add_scatter(x=pca["0"], 
    y=pca["1"], 
    marker_color=list(pca.colors),
    mode='markers',
    marker=dict(size=10, opacity=0.2),
    text=list(pca.ID + ', ' + pca['Class'])
)

fig.update_layout(
    title="PCA",
    xaxis_title="PC1",
    yaxis_title="PC2",
    width=800, height=800
    )


fig1 = fig.data[0]


prob = pd.concat([pca['ID'], pca['Class'], pca.filter(regex='Prob_', axis=1)], axis=1)
prob = round(prob, 2)

colnames = list(time_series.columns.values)
colnames.remove('ID')
colnames.remove('class')
groups = list(OrderedDict.fromkeys([i.split('_')[0] for i in colnames]))


measurements = []

for i in groups:
    measurements.append(time_series.filter(regex=i))



    
out = Output()
@out.capture(clear_output=True)
def display_timeseries(trace, points, state):
    
    ts = make_subplots(rows=2, cols=2,  
        subplot_titles=['PCA', 'ID: ' + time_series.loc[points.point_inds[0],'ID'] + ' -> Class: '+ str(time_series.loc[points.point_inds[0],'class'])],
        specs=[[{"type": "scatter",'rowspan': 2},
            {"type": "scatter"}],
            [None,
            {"type": "table"}]],
        row_width=[0.1, 0.5]
        )
    for i in range(len(measurements)):
        #ts.add_trace(go.Scatter(x=list(range(0,(time_series.shape[1]-2)*5,5)),
        #    y=time_series.iloc[points.point_inds[0],2:time_series.shape[1]],
        #    marker_color=pca.colors[points.point_inds[0]]
        #    ),row=1,col=2) marker_color=float('0.'+str(i))+0.1
        ts.add_trace(go.Scatter(x=list(range(0,(measurements[i].shape[1])*5,5)),
            y=measurements[i].iloc[points.point_inds[0],0:measurements[i].shape[1]],
            name=groups[i]
            ),row=1,col=2)

    ts.add_trace(go.Scatter(x=pca["0"], 
        y=pca["1"], 
        marker_color=list(pca.colors),
        mode='markers',
        marker=dict(size=10, opacity=0.5),
        text=list(pca.ID+', '+pca['Class'])
    ), row=1, col=1)

    table=go.Figure(data=[go.Table(header=dict(values=list(prob.columns)),
                cells=dict(values=[prob[col][points.point_inds[0]] for col in prob.columns]))])
    ts.add_trace(table.data[0], row=2, col=2)
    
    
    ts.add_trace(go.Scatter(x=points.xs, 
        y=points.ys, 
        marker_color='red',
        mode='markers',
        marker=dict(size=10, opacity=1)
    ), row=1, col=1)
    
    
    ts['layout']['xaxis'].update(title_text='PC1')
    ts['layout']['yaxis'].update(title_text='PC2')
    
    ts['layout']['xaxis2'].update(title_text='Time in minutes')
    ts['layout']['yaxis2'].update(title_text='Ratio')
    
    ts.update_layout(title='Table and Scatter Plot from Pandas DataFrame')

    tsf = go.FigureWidget(ts)
    
    tsf.write_html("./ts.html")
    plot_file ='ts.html'
    
    def serve_plot():
        handler = http.server.SimpleHTTPRequestHandler
        with socketserver.TCPServer(("", 0), handler) as httpd:
            port = httpd.server_address[1]
            webbrowser.open(f"http://localhost:{port}/{plot_file}")
            httpd.serve_forever()

    server_thread = threading.Thread(target=serve_plot)
    server_thread.daemon = True
    server_thread.start()

    import time
    time.sleep(1)

    return f"http://localhost:{port}/{plot_file}"



fig1.on_click(display_timeseries)

VBox([fig, out])


    'data': [{'marker': {'color': [green, green, green, ..., blue, blue, blue],
…